# Logistic regression to predict tip category

Obtain necessary data from the two datafiles:

In [31]:
import csv
infile = open("nyc_data.csv", 'r')    # open the file for reading
infile2 = open("nyc_fare.csv", 'r')

reader = csv.reader(infile)
reader2 = csv.reader(infile2)
reader.next() #remove headers
reader2.next()

taxi_file = []
taxi_file2 = []

for line in reader:     
    taxi_file.append(line)
    
nb = 0
for line2 in reader2:                   # read through the CSV one line at a time
    taxi_file2.append(line2)
    
print len(taxi_file)

846945


In [32]:
temp =[]
data = []

for line in taxi_file:
    data_list = []
    data_list.append(float(line[7]))
    data_list.append(float(line[8]))
    data_list.append(float(line[9]))
    temp.append(data_list)
    
nb = 0
for line2 in taxi_file2:                   # read through the CSV one line at a time
    if line2[4] == 'CRD':
        data.append(temp[nb]+ ([float(line2[5]) + float(line2[6]) + float(line2[7]) + float(line2[9])] + ([line2[8]])))
    nb+=1
    
print len(data)
print data[0]

456019
[1.0, 960.0, 3.28, 14.5, '2.8']


Transform tip value in categories and normalize numerical values:

In [33]:
import numpy as np
from __future__ import division

x_mean = np.array([line[:4] for line in data]).mean(axis=0)
x_std = np.array([line[:4] for line in data]).std(axis=0)

y_mean = np.mean([float(d[4]) for d in data])
y_low = y_mean * 2/5
y_medium_low = y_low * 2
y_medium = y_mean * 6/5
y_medium_high = y_mean * 8/5

print y_low
print y_medium_low
print y_medium
print y_medium_high

i=0
for d in data:
    data[i][0] = (d[0] - x_mean[0])/x_std[0]
    data[i][1] = (d[1] - x_mean[1])/x_std[1]
    data[i][2] = (d[2] - x_mean[2])/x_std[2]
    data[i][3] = (d[3] - x_mean[3])/x_std[3]
    if float(d[4]) < y_low:
        data[i][4] = 'low'
    elif float(d[4]) < y_medium_low:
        data[i][4] = 'medium-low'
    elif float(d[4]) < y_medium:
        data[i][4] = 'medium'
    elif float(d[4]) < y_medium_high:
        data[i][4] = 'medium-high'
    else:
        data[i][4] = 'high'
    i+=1 

0.996702277756
1.99340455551
2.99010683327
3.98680911102


Divide in training and testing set:

In [34]:
import random

train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

train  364900
test  91119


Get input and output variables for the training set:

In [35]:
x = [line[:4] for line in train]
y = [line[4] for line in train]

Perform multivariate regression:

In [36]:
from sklearn import datasets, linear_model

regr = linear_model.LogisticRegression()
regr.fit(x, y)

print'Coefficients: ', regr.coef_

Coefficients:  [[ -2.10517021e-02  -3.54842330e-03  -3.47314216e-03   2.60222124e+00]
 [  5.13792047e-02   3.19369123e-03  -6.65188665e-03  -2.22349328e-01]
 [ -6.23260445e-03   4.74636826e-03   9.74433832e-03  -2.54902155e-01]
 [ -3.42967358e-03   3.76919278e-03  -6.23007388e-03   2.17493061e-01]
 [  6.09494690e-03  -5.34665271e-03  -1.03110534e-02  -3.46317581e+00]]


Prepare the testing set:

In [37]:
x_test = [line[:4] for line in test]
y_test = [line[4] for line in test]

Get prediction for the testing set:

In [38]:
prediction = regr.predict(x_test)

Get performance score on testing set:
    

In [39]:
print regr.score(x_test, y_test)

0.581931320581


The $r^2$ score is not that bad, but there is room for improvement. Maybe the logisitic regression approach is not the best: let's try with a decision tree classifier.

For this, we will wome back to the first dataset, using the number of passengers, the trip duration, the trip distance, the amount of the trip, based only on trips paid with credit cards.

Now let's fit a decision tree to the training set:

In [40]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Now let's get the score of the decision tree for the testing set:

In [41]:
print clf.score(x_test, y_test)

0.594102217979


We could also try with a random forest classifier:

In [42]:
from sklearn import ensemble

forest = ensemble.RandomForestClassifier()
forest.fit(x,y)
forest.score(x_test, y_test)

0.63198674261131049

The $r^2$ score hasn't been improved so much, but there is a nice improvement with a random forst classifier.Now, maybe we can try to add different features, such as the day of the week and the time of the day.

In [43]:
from sklearn import preprocessing
import datetime

weekdays = []
hours = []

for d in taxi_file2:
    if d[4] == 'CRD':
        daytime = d[3]
        date, time = daytime.split(" ")
        year, month, day = date.split("-")
        hour, minute, second = time.split(":")
        my_date = datetime.date(int(year), int(month), int(day))
        weekdays.append(my_date.weekday())
        hours.append(hour)  
    

lb = preprocessing.LabelBinarizer()
lb2 = preprocessing.LabelBinarizer()

hours = lb.fit_transform(hours).tolist()
weekdays = lb2.fit_transform(weekdays).tolist()

nb = 0

for d in data:
    data[nb] = data[nb] + hours[nb]
    data[nb] = data[nb] + weekdays[nb]
    nb += 1
    
print data[0]

[-0.49952274252615503, 0.0069147838584908842, -0.0014624231423751382, 0.036605255018071575, 'medium', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


Now that we have a new dataset to work with, we can create new training and testing sets, and try a logisitic regression, a decision tree, and a random forest classifier.

In [44]:
import random

train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

x = [line[:4] + line[5:] for line in train]
y = [line[4] for line in train]

x_test = [line[:4]+ line[5:] for line in test]
y_test = [line[4] for line in test]

regr = linear_model.LogisticRegression()
regr.fit(x, y)
print 'R2 for Logistic Regression: ', regr.score(x_test, y_test)

clf = tree.DecisionTreeClassifier()
clf.fit(x, y)
print 'R2 for Decision Tree: ', clf.score(x_test, y_test) 

forest = ensemble.RandomForestClassifier()
forest.fit(x,y)
print 'R2 for Random Forest Classifier: ', forest.score(x_test, y_test)

train  364987
test  91032
R2 for Logistic Regression:  0.577796818701
R2 for Decision Tree:  0.565021091484
R2 for Random Forest Classifier:  0.636863959926


By takink the pick-up time into account, we can achieve a $r^2$ score of 68% with a random forest classifier; which is now pretty good. However, we could try one last modification by taking the pick-up and drop-off locations into account.

As usual, let's start by creating a new dataset to work with:

In [45]:
from sklearn import preprocessing
import datetime

locations = []
nb = 0

for d in taxi_file2:
    if d[4] == 'CRD':
        locations.append([float(taxi_file[nb][10])] + [float(taxi_file[nb][11])] + [float(taxi_file[nb][12])] + [float(taxi_file[nb][13])])
        nb+=1
        
print len(locations)
print locations[0]

nb = 0

for d in data:
    data[nb] = data[nb] + locations[nb]
    nb += 1
    
print data[0]

456019
[-73.955925, 40.781887, -73.963181, 40.777832000000004]
[-0.49952274252615503, 0.0069147838584908842, -0.0014624231423751382, 0.036605255018071575, 'medium', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -73.955925, 40.781887, -73.963181, 40.777832000000004]


And now let's look at the scores for the different methods:

In [46]:
import random

train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

x = [line[:4] + line[5:] for line in train]
y = [line[4] for line in train]

x_test = [line[:4]+ line[5:] for line in test]
y_test = [line[4] for line in test]

regr = linear_model.LogisticRegression()
regr.fit(x, y)
print 'R2 for Logistic Regression: ', regr.score(x_test, y_test)

clf = tree.DecisionTreeClassifier()
clf.fit(x, y)
print 'R2 for Decision Tree: ', clf.score(x_test, y_test) 

forest = ensemble.RandomForestClassifier()
forest.fit(x,y)
print 'R2 for Random Forest Classifier: ', forest.score(x_test, y_test)

train  365025
test  90994
R2 for Logistic Regression:  0.578554629976
R2 for Decision Tree:  0.561751324263
R2 for Random Forest Classifier:  0.673934545135


After comparing the different techniques and the different attributes that could be taken into account, it looks like the best we can achieve is 67% with a random forest classifier taking into account all the attributes we have.

This is not bad, and it might not be possible to improve more because we would need data that we don't have access to: the amount of the tip might depend a lot on the client's social background such as his age, his profession, his marital status...